# General aim of this Notebook :

The general aim of this notebook is to find the best hyperparameters for the Convolutional Neural Network, using GridSearchCV. 

In order to use it, one needs to load the training datas. 

The best hyperparameters found are indicated.

In [57]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split 

# Load the data

In [106]:
data_train_path = 'C:/Users/Flore.MININT-64PJ14C/Documents/GitHub/ML_Project2/results/Split_400_0.8/Combined_All_Train_80.csv'
tx_train = pd.read_csv(data_train_path,sep=",",squeeze=True)
X_train=tx_train.iloc[:, :400]
ytr = pd.read_csv(data_train_path,sep=",",usecols=[400],squeeze=True)
y_train = tf.keras.utils.to_categorical(ytr,num_classes=14)

In [107]:
print(X_train.shape)
print(y_train.shape)


(10908, 400)
(10908, 14)


# Finding the hyperparameters


In [60]:
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [108]:
n_samples, n_features = X_train.shape[0], X_train.shape[1]
n_outputs=14

In [74]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [109]:
print(X_train.shape)
X_train=np.expand_dims(X_train, axis=-1)
print(X_train.shape)


(10908, 400)
(10908, 400, 1)


In [54]:
#define the general model
model = tf.keras.Sequential()
model.add(Conv1D(filters=12, kernel_size=5, activation='relu',input_shape=(n_features, 1)))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(8, input_dim=199, activation='relu'))
model.add(Dense(14, activation='softmax'))
#Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Find best batch size and number of epoch, BEST: bs=1 and epoch=10

In [83]:
def create_model():
    # create model
    model = tf.keras.Sequential()
    model.add(Conv1D(filters=12, kernel_size=5, activation='relu',input_shape=(n_features, 1)))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(8, input_dim=199, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    #Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [88]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [1, 10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.154132 using {'batch_size': 1, 'epochs': 10}
0.154132 (0.073810) with: {'batch_size': 1, 'epochs': 10}
0.079439 (0.039836) with: {'batch_size': 1, 'epochs': 50}
0.106365 (0.039468) with: {'batch_size': 1, 'epochs': 100}
0.108532 (0.046159) with: {'batch_size': 10, 'epochs': 10}
0.103889 (0.041429) with: {'batch_size': 10, 'epochs': 50}
0.090994 (0.030404) with: {'batch_size': 10, 'epochs': 100}
0.086557 (0.039219) with: {'batch_size': 20, 'epochs': 10}
0.097493 (0.012256) with: {'batch_size': 20, 'epochs': 50}
0.098731 (0.046335) with: {'batch_size': 20, 'epochs': 100}
0.102342 (0.046140) with: {'batch_size': 40, 'epochs': 10}
0.057670 (0.049711) with: {'batch_size': 40, 'epochs': 50}
0.146497 (0.088628) with: {'batch_size': 40, 'epochs': 100}
0.083565 (0.041869) with: {'batch_size': 60, 'epochs': 10}
0.070670 (0.055715) with: {'batch_size': 60, 'epochs': 50}
0.095017 (0.024961) with: {'batch_size': 60, 'epochs': 100}
0.102032 (0.039122) with: {'batch_size': 80, 'epochs': 10}
0

## Find best optimizer, BEST : Nadam

In [89]:
def create_model(optimizer='adam'):
    # create model
    model = tf.keras.Sequential()
    model.add(Conv1D(filters=12, kernel_size=5, activation='relu',input_shape=(n_features, 1)))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(8, input_dim=199, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [90]:
#create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.123285 using {'optimizer': 'Nadam'}
0.098937 (0.029671) with: {'optimizer': 'SGD'}
0.040751 (0.044148) with: {'optimizer': 'RMSprop'}
0.075003 (0.065321) with: {'optimizer': 'Adagrad'}
0.087795 (0.067563) with: {'optimizer': 'Adadelta'}
0.041164 (0.058214) with: {'optimizer': 'Adam'}
0.069328 (0.028388) with: {'optimizer': 'Adamax'}
0.123285 (0.090091) with: {'optimizer': 'Nadam'}


## Find best learning rate, BEST : 0.01

In [93]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = tf.keras.Sequential()
    model.add(Conv1D(filters=12, kernel_size=5, activation='relu',input_shape=(n_features, 1)))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(8, input_dim=199, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Nadam(lr=learn_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model



In [94]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.136078 using {'learn_rate': 0.01}
0.093882 (0.043343) with: {'learn_rate': 0.001}
0.136078 (0.085941) with: {'learn_rate': 0.01}
0.000000 (0.000000) with: {'learn_rate': 0.1}
0.083153 (0.117596) with: {'learn_rate': 0.2}
0.026308 (0.037205) with: {'learn_rate': 0.3}


## Find best Network Weight Initialization, BEST: he_uniform

In [112]:
def create_model(init_mode='uniform'):
    # create model
    model = tf.keras.Sequential()
    model.add(Conv1D(filters=12, kernel_size=5, activation='relu',kernel_initializer=init_mode,input_shape=(n_features, 1)))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu',kernel_initializer=init_mode))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(8, input_dim=199, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Nadam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [113]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.104969 using {'init_mode': 'he_uniform'}
0.045013 (0.063658) with: {'init_mode': 'uniform'}
0.083792 (0.118499) with: {'init_mode': 'lecun_uniform'}
0.083792 (0.118499) with: {'init_mode': 'normal'}
0.083792 (0.118499) with: {'init_mode': 'zero'}
0.083792 (0.118499) with: {'init_mode': 'glorot_normal'}
0.083792 (0.118499) with: {'init_mode': 'glorot_uniform'}
0.083792 (0.118499) with: {'init_mode': 'he_normal'}
0.104969 (0.106724) with: {'init_mode': 'he_uniform'}


## Find best Neuron Activation Function, BEST: 'tanh'

In [114]:
def create_model(activation='relu'):
    # create model
    model = tf.keras.Sequential()
    model.add(Conv1D(filters=12, kernel_size=5, activation=activation,kernel_initializer='he_uniform',input_shape=(n_features, 1)))
    model.add(Conv1D(filters=64, kernel_size=5, activation=activation,kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(8, input_dim=199, activation=activation))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Nadam(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [115]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=0)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.101393 using {'activation': 'tanh'}
0.083792 (0.118499) with: {'activation': 'softmax'}
0.083792 (0.118499) with: {'activation': 'softplus'}
0.066832 (0.032525) with: {'activation': 'softsign'}
0.083792 (0.118499) with: {'activation': 'relu'}
0.101393 (0.108222) with: {'activation': 'tanh'}
0.083792 (0.118499) with: {'activation': 'sigmoid'}
0.042171 (0.059639) with: {'activation': 'hard_sigmoid'}
0.085900 (0.046808) with: {'activation': 'linear'}


## Find best Dropout Regularization + weight constraint

In [123]:
#from keras.constraints import maxnorm

def create_model(dropout_rate=0.5):#, weight_constraint=0):
    # create model
    model = tf.keras.Sequential()
    model.add(Conv1D(filters=12, kernel_size=5, activation='tanh',kernel_initializer='he_uniform',input_shape=(n_features, 1))) # kernel_constraint=maxnorm(weight_constraint),
    model.add(Conv1D(filters=64, kernel_size=5, activation='tanh',kernel_initializer='he_uniform' ))
    model.add(Dropout(dropout_rate))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(8, input_dim=199, activation='tanh'))
    model.add(Dense(14, activation='softmax'))
    # Compile model
    optimizer = tf.keras.optimizers.Nadam(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [124]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=0)

# define the grid search parameters
#weight_constraint = [0,1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate)#, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.134213 using {'dropout_rate': 0.0}
0.134213 (0.114341) with: {'dropout_rate': 0.0}
0.042171 (0.059639) with: {'dropout_rate': 0.1}
0.096535 (0.110171) with: {'dropout_rate': 0.2}
0.078841 (0.062316) with: {'dropout_rate': 0.3}
0.122937 (0.101871) with: {'dropout_rate': 0.4}
0.057297 (0.022224) with: {'dropout_rate': 0.5}
0.000000 (0.000000) with: {'dropout_rate': 0.6}
0.122021 (0.102754) with: {'dropout_rate': 0.7}
0.041896 (0.058667) with: {'dropout_rate': 0.8}
0.125504 (0.091622) with: {'dropout_rate': 0.9}


## Find best Number of Neurons in the Hidden Layer, BEST: 105

In [136]:
def create_model(neurons=8):
    # create model
    model = tf.keras.Sequential()
    model.add(Conv1D(filters=12, kernel_size=5, activation='tanh',kernel_initializer='he_uniform',input_shape=(n_features, 1)))
    model.add(Conv1D(filters=64, kernel_size=5, activation='tanh',kernel_initializer='he_uniform'))
    model.add(Dropout(0.4))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(neurons, input_dim=199, activation='tanh'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Nadam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [137]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1, verbose=0)

# define the grid search parameters
neurons = [95, 100, 105]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.118995 using {'neurons': 105}
0.102494 (0.056236) with: {'neurons': 95}
0.077833 (0.013030) with: {'neurons': 100}
0.118995 (0.074054) with: {'neurons': 105}
